In [1]:
import os
from ultralytics import YOLO
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
# Run model accuracy, precision, and recall
model = YOLO("./best/best.pt")
image_folder = './datasets/test/images'
label_folder = './datasets/test/labels'

true_labels = []
predicted_labels = []
for image_file in os.listdir(image_folder):
    base_filename = os.path.splitext(image_file)[0]
    label_file_path = os.path.join(label_folder, f"{base_filename}.txt")
    
    if not os.path.exists(label_file_path):
        continue
    
    with open(label_file_path, 'r') as label_file:
        true_class_id = int(label_file.readline().strip().split()[0])
        true_labels.append(true_class_id)
    
    image_path = os.path.join(image_folder, image_file)
    results = model.predict(source=image_path, save=False, save_txt=False)
    
    highest_confidence = -1
    best_class_id = -1  # Use -1 if no predictions found
    
    if results:
        for result in results:
            for detection in result.boxes.data:
                confidence = float(detection[4])  
                class_id = int(detection[5])      
                if confidence > highest_confidence:
                    highest_confidence = confidence
                    best_class_id = class_id
    
    predicted_labels.append(best_class_id)

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=1)
recall = recall_score(true_labels, predicted_labels, average='weighted', zero_division=1)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")